 first we define needed dataset

In [ ]:
import numpy as np
X_train = np.array([
    [0, 1, 1],
    [0, 0, 1],
    [0, 0, 0],
    [1, 1, 0]])
Y_train = ['Y', 'N', 'Y', 'Y']
X_test = np.array([[1, 1, 0]])

 function wicth based on label we group the data

In [ ]:
def get_label_indices(labels):
  from collections import defaultdict
  label_indices = defaultdict(list)
  for index, label in enumerate(labels):
    label_indices[label].append(index)
  return label_indices

in this place we can see data labeled and it result

In [ ]:
label_indices = get_label_indices(Y_train)
print('label_indices:\n', label_indices)

label_indices:
 defaultdict(<class 'list'>, {'Y': [0, 2, 3], 'N': [1]})


this function calculate prior

In [ ]:
def get_prior(label_indices):
  prior = {label: len(indices) for label, indices in label_indices.items()}
  total_count = sum(prior.values())
  for label in prior:
    prior[label] /= total_count
  return prior

now lets compute prior

In [ ]:
prior = get_prior(label_indices)
print('Prior:', prior)

Prior: {'Y': 0.75, 'N': 0.25}


With prior calculated, we continue with likelihood, which is the conditional probability

In [ ]:
def get_likelihood(features, label_indices, smoothing=0):
  likelihood = {}
  for label, indices in label_indices.items():
    likelihood[label] = features[indices, :].sum(axis=0)+ smoothing
    total_count = len(indices)
    likelihood[label] = likelihood[label] / (total_count + 2 * smoothing)
  return likelihood

We set the smoothing value to 1 here, which can also be 0 for no smoothing, or any other positive value, as long as a higher classification performance is achieved:

In [ ]:
smoothing = 1
likelihood = get_likelihood(X_train, label_indices, smoothing)
print('Likelihood:\n', likelihood)

Likelihood:
 {'Y': array([0.4, 0.6, 0.4]), 'N': array([0.33333333, 0.33333333, 0.66666667])}


With prior and likelihood ready, we can now compute the posterior for the testing/ new samples:

In [ ]:
def get_posterior(X, prior, likelihood):
  posteriors = []
  for x in X:
    posterior = prior.copy()
    for label, likelihood_label in likelihood.items():
      for index, bool_value in enumerate(x):
        posterior[label] *= likelihood_label[index] if bool_value else (1 - likelihood_label[index])
      sum_posterior = sum(posterior.values())
      for label in posterior:
        if posterior[label] == float('inf'):
          posterior[label] = 1.0
        else:
          posterior[label] /= sum_posterior
      posteriors.append(posterior.copy())
    return posteriors



Now, let's predict the class of our one sample test set using this prediction function:

In [ ]:
posterior = get_posterior(X_test, prior, likelihood)
print('Posterior:\n', posterior)

Posterior:
 [{'Y': 0.30167597765363136, 'N': 0.6983240223463687}, {'Y': 0.9210360075805433, 'N': 0.0789639924194567}]
